In [2]:
import h5py
import numpy as np
import os
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
import pandas as pd
from keras import backend as K

import tensorflow as tf
from tensorflow.python.tools import freeze_graph
from tensorflow.python.tools import optimize_for_inference_lib

Using TensorFlow backend.


In [3]:
MODEL_NAME = 'arabic_digits_classifier_keras'
EPOCHS = 4
BATCH_SIZE = 200
IMAGE_SIZE = 28
NUM_CLASSES = 10

In [4]:
TRAIN_DIR = "E:\\DataSets\\MAHDBase_TrainingSet\\TrainingSet.csv"
TEST_DIR = "E:\\DataSets\\MAHDBase_TrainingSet\\TestSet.csv"
LABELS_DIR = "E:\\DataSets\\MAHDBase_TrainingSet\\TrainingLabelsOnehot.csv"
TEST_LABELS_DIR = "E:\\DataSets\\MAHDBase_TrainingSet\\TestLabelsOnehot.csv"
data_x= pd.read_csv(TRAIN_DIR, header= None)
data_test = pd.read_csv(TEST_DIR, header= None)
data_y = pd.read_csv(LABELS_DIR, header= None)
data_y_test = pd.read_csv(TEST_LABELS_DIR, header= None)

#X = np.genfromtxt (TRAIN_DIR, delimiter=',')
X = np.array(data_x.values)
Y = np.array(data_y.values)
X_test = np.array(data_test.values)
Y_test = np.array(data_y_test.values)

In [17]:
def prepare_data():

    x_train = X.reshape(X.shape[0], IMAGE_SIZE, IMAGE_SIZE, 1)
    x_test = X_test.reshape(X_test.shape[0], IMAGE_SIZE, IMAGE_SIZE, 1)
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    x_train /= 255
    x_test /= 255
    y_train = Y
    y_test = Y_test
    return x_train, y_train, x_test, y_test


In [7]:
def build_model():
    model = Sequential()
    model.add(Conv2D(filters=64, kernel_size=3, strides=1, padding='same', activation='relu', input_shape=[IMAGE_SIZE, IMAGE_SIZE, 1]))
    # 28*28*64
    model.add(MaxPooling2D(pool_size=2, strides=2, padding='same'))
    # 14*14*64

    model.add(Conv2D(filters=128, kernel_size=3, strides=1, padding='same', activation='relu'))
    # 14*14*128
    model.add(MaxPooling2D(pool_size=2, strides=2, padding='same'))
    # 7*7*128

    model.add(Conv2D(filters=256, kernel_size=3, strides=1, padding='same', activation='relu'))
    # 7*7*256
    model.add(MaxPooling2D(pool_size=2, strides=2, padding='same'))
    # 4*4*256

    model.add(Flatten())
    model.add(Dense(1024, activation='relu'))
    #model.add(Dropout(0.5))
    model.add(Dense(10, activation='softmax'))
    return model

In [8]:
def train(model, x_train, y_train, x_test, y_test):
    model.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adadelta(),  metrics=['accuracy'])
    model.fit(x_train, y_train, batch_size=BATCH_SIZE, epochs=EPOCHS, verbose=1, validation_data=(x_test, y_test))

In [9]:
def export_model(saver, model, input_node_names, output_node_name):
    model.save(MODEL_NAME +'.h5')
    tf.train.write_graph(K.get_session().graph_def, 'out', \
        MODEL_NAME + '_graph.pbtxt')

    saver.save(K.get_session(), 'out/' + MODEL_NAME + '.chkp')

    freeze_graph.freeze_graph('out/' + MODEL_NAME + '_graph.pbtxt', None, \
        False, 'out/' + MODEL_NAME + '.chkp', output_node_name, \
        "save/restore_all", "save/Const:0", \
        'out/frozen_' + MODEL_NAME + '.pb', True, "")

    input_graph_def = tf.GraphDef()
    with tf.gfile.Open('out/frozen_' + MODEL_NAME + '.pb', "rb") as f:
        input_graph_def.ParseFromString(f.read())

    output_graph_def = optimize_for_inference_lib.optimize_for_inference(
            input_graph_def, input_node_names, [output_node_name],
            tf.float32.as_datatype_enum)

    with tf.gfile.FastGFile('out/opt_' + MODEL_NAME + '.pb', "wb") as f:
        f.write(output_graph_def.SerializeToString())

    print("graph saved!")




In [10]:
def main():
    if not os.path.exists('out'):
        os.mkdir('out')

    x_train, y_train, x_test, y_test = prepare_data()

    model = build_model()

    train(model, x_train, y_train, x_test, y_test)

    export_model(tf.train.Saver(), model, ["conv2d_1_input"], "dense_2/Softmax")

In [11]:
main()

Train on 55000 samples, validate on 5000 samples
Epoch 1/4
55000/55000 [==============================] - 510s 9ms/step - loss: 0.7186 - acc: 0.7861 - val_loss: 0.1400 - val_acc: 0.9602
Epoch 2/4
55000/55000 [==============================] - 498s 9ms/step - loss: 0.0725 - acc: 0.9789 - val_loss: 0.0725 - val_acc: 0.9764
Epoch 3/4
55000/55000 [==============================] - 494s 9ms/step - loss: 0.0485 - acc: 0.9857 - val_loss: 0.0744 - val_acc: 0.9768
Epoch 4/4
55000/55000 [==============================] - 493s 9ms/step - loss: 0.0397 - acc: 0.9880 - val_loss: 0.0548 - val_acc: 0.9840
INFO:tensorflow:Restoring parameters from out/arabic_digits_classifier_keras.chkp
INFO:tensorflow:Froze 10 variables.
Converted 10 variables to const ops.
graph saved!
